In [16]:
# 1) (Run once)
#    %pip install arcgis geopandas pandas tqdm

from arcgis.gis import GIS
import os, zipfile
import geopandas as gpd

# 2) Connect & prepare paths
gis      = GIS()  # anonymous
item_id  = "4de9d717f1d548e0bfaee3fefdf64b51"   # Beirut Parcel dataset
raw_dir  = "datasets/beirut_buildings"
os.makedirs(raw_dir, exist_ok=True)

# 3) Get the item and export it
item        = gis.content.get(item_id)
export_job  = item.export("beirut_parcels_export", "Shapefile")
export_item = export_job  # this is now the exported Item

# 4) Download the ZIP
zip_path = os.path.join(raw_dir, "beirut_parcels.zip")
export_item.download(save_path=zip_path)
print("Downloaded ZIP →", zip_path)

# 5) Unzip into the same folder
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(raw_dir)
print("Unzipped to →", raw_dir)

# 6) (Optional) Inspect with GeoPandas
shp_files = [f for f in os.listdir(raw_dir) if f.endswith(".shp")]
print("Found shapefiles:", shp_files)
gdf = gpd.read_file(os.path.join(raw_dir, shp_files[0]))
print(f"Loaded {len(gdf):,} parcels; CRS = {gdf.crs}")


c:\Users\charb\AppData\Local\Programs\Python\Python312\Lib\site-packages\arcgis\gis\__init__.py:6071: SyntaxWarning: invalid escape sequence '\s'
  """
c:\Users\charb\AppData\Local\Programs\Python\Python312\Lib\site-packages\arcgis\gis\__init__.py:9679: SyntaxWarning: invalid escape sequence '\{'
  new_path = f"{current_path}\{title}" if current_path else title
c:\Users\charb\AppData\Local\Programs\Python\Python312\Lib\site-packages\arcgis\gis\__init__.py:14390: SyntaxWarning: invalid escape sequence '\A'
  """
c:\Users\charb\AppData\Local\Programs\Python\Python312\Lib\site-packages\arcgis\gis\_impl\_con\_connection.py:1032: SyntaxWarning: invalid escape sequence '\m'
  """
c:\Users\charb\AppData\Local\Programs\Python\Python312\Lib\site-packages\arcgis\gis\_impl\_con\_connection.py:1285: SyntaxWarning: invalid escape sequence '\m'
  """


AttributeError: 'NoneType' object has no attribute 'username'

In [14]:
# ╔════════════════════════════════════════════════════════════════════════╗
# ║  ONE-SHOT BEIRUT WRANGLER → 256×256 PNGs READY FOR PIX2PIX            ║
# ╚════════════════════════════════════════════════════════════════════════╝
import os, shutil, math, json
import geopandas as gpd
import numpy as np
import cv2
from shapely.affinity import translate
from shapely.geometry import Polygon, box
from tqdm import tqdm

# ─── paths ───────────────────────────────────────────────────────────────
RAW = "datasets/beirut_buildings"      # where .shp files live
OUT = "data/processed/beirut_pix2pix"           # final pix2pix root
os.makedirs(OUT, exist_ok=True)

# ─── load shapefiles ─────────────────────────────────────────────────────
parcels   = gpd.read_file(f"{RAW}/beirut_parcels.shp").to_crs(epsg=3857)
footprints= gpd.read_file(f"{RAW}/beirut_buildings.shp").to_crs(epsg=3857)

# ─── output folders ──────────────────────────────────────────────────────
folder_A = os.path.join(OUT, "A")   # parcel masks
folder_B = os.path.join(OUT, "B")   # footprint masks
for p in (folder_A, folder_B):
    shutil.rmtree(p, ignore_errors=True)
    os.makedirs(p)

# ─── helper: rasterise polygon list to 256×256 mask ──────────────────────
def rasterize_layer(loops, height, width, pad=5, color=(255, 255, 255), rgb=False):
    """
    Rasterize polygons to 256×256 mask.
    - color: grayscale int or RGB tuple
    - rgb: if True, returns 3-channel RGB image
    """
    if not loops:
        return np.zeros((256, 256, 3), np.uint8) if rgb else np.zeros((256, 256), np.uint8)

    clean = [[(pt[0], pt[1]) for pt in loop] for loop in loops]
    minx = min(x for loop in clean for x, _ in loop)
    miny = min(y for loop in clean for _, y in loop)
    clean = [[(x - minx, y - miny) for x, y in loop] for loop in clean]

    scale = max(height, width) + pad * 2
    img = np.zeros((256, 256, 3), np.uint8) if rgb else np.zeros((256, 256), np.uint8)
    for loop in clean:
        pts = np.array(
            [[(x + pad) / scale * 255, (y + pad) / scale * 255] for x, y in loop],
            np.int32,
        )
        if rgb:
            cv2.fillPoly(img, [pts], color)
        else:
            cv2.fillPoly(img, [pts], color)
    return img

# ─── iterate parcels, find building loops, save PNGs ─────────────────────
meta = []
for idx, parc in tqdm(list(parcels.iterrows()), desc="Rasterising"):
    if parc.geometry is None or parc.geometry.is_empty:
        continue

    sel = footprints[footprints.geometry.intersects(parc.geometry)]
    if sel.empty:
        continue

    bldg_loops = [list(poly.exterior.coords) for poly in sel.geometry if isinstance(poly, Polygon)]

    parcel_loop = []
    if isinstance(parc.geometry, Polygon):
        parcel_loop.append(list(parc.geometry.exterior.coords))
    else:
        for poly in parc.geometry.geoms:
            parcel_loop.append(list(poly.exterior.coords))

    minx, miny, maxx, maxy = parc.geometry.bounds
    h = int(maxx - minx)
    w = int(maxy - miny)

    # A: binary mask (white parcel on black)
    img_A = rasterize_layer(parcel_loop, h, w, color=255, rgb=False)

    # B: RGB image with parcels as gray, buildings as red
    img_B = rasterize_layer(parcel_loop, h, w, color=(200, 200, 200), rgb=True)
    building_mask = rasterize_layer(bldg_loops, h, w, color=(255, 0, 0), rgb=True)

    # Overwrite red buildings on top of gray parcel background
    red_mask = (building_mask[:, :, 0] == 255) & (building_mask[:, :, 1] == 0) & (building_mask[:, :, 2] == 0)
    img_B[red_mask] = (255, 0, 0)

    uid = f"beirut_{idx:05d}.png"
    cv2.imwrite(os.path.join(folder_A, uid), img_A)
    cv2.imwrite(os.path.join(folder_B, uid), img_B)
    meta.append(uid)

print(f"✅ Wrote {len(meta)} PNG pairs → {folder_A} & {folder_B}")

# ─── quick train/val/test split & pix2pix pairing ────────────────────────
np.random.shuffle(meta)
n = len(meta)
splits = {
    "train": meta[int(0.10*n):],
    "val":   meta[:int(0.05*n)],
    "test":  meta[int(0.05*n):int(0.10*n)],
}
pair_root = os.path.join(OUT, "parcel__footprint_AB")
for split, files in splits.items():
    dst = os.path.join(pair_root, split)
    os.makedirs(dst, exist_ok=True)
    for fname in files:
        A = cv2.imread(os.path.join(folder_A, fname))
        B = cv2.imread(os.path.join(folder_B, fname))
        AB = np.concatenate([A, B], axis=1)
        cv2.imwrite(os.path.join(dst, fname), AB)

print("🖼️  Pix2pix A⧸B pairs ready under:", pair_root)


c:\Users\charb\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: datasets/beirut_buildings/beirut_buildings.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
Rasterising: 100%|██████████| 21143/21143 [02:19<00:00, 151.75it/s]


✅ Wrote 18121 PNG pairs → data/processed/beirut_pix2pix\A & data/processed/beirut_pix2pix\B


KeyboardInterrupt: 

In [15]:
#THIS IS THE RIGHT ONE
from shapely.geometry import Polygon, MultiPolygon

# ─── Settings ────────────────────────────────────────────────────────────────
PAD = 5
SIZE = 256
meta = []

# ─── Rasterize each parcel + clipped buildings in red ────────────────────────
for idx, parc in tqdm(parcels.iterrows(), desc="Rasterising", total=len(parcels)):
    if parc.geometry is None or parc.geometry.is_empty:
        continue

    # select and clean
    sel = footprints[footprints.geometry.intersects(parc.geometry)]
    if sel.empty:
        continue
    fixed_parcel = parc.geometry.buffer(0)

    # collect parcel exterior loops
    parcel_loops = []
    if isinstance(fixed_parcel, Polygon):
        parcel_loops.append(list(fixed_parcel.exterior.coords))
    else:
        for part in fixed_parcel.geoms:
            parcel_loops.append(list(part.exterior.coords))

    # clip & collect building loops
    bldg_loops = []
    for poly in sel.geometry:
        try:
            fixed_bldg = poly.buffer(0)
            clipped = fixed_bldg.intersection(fixed_parcel)
            if clipped.is_empty:
                continue
            if isinstance(clipped, Polygon):
                bldg_loops.append(list(clipped.exterior.coords))
            elif isinstance(clipped, MultiPolygon):
                for sub in clipped.geoms:
                    bldg_loops.append(list(sub.exterior.coords))
        except Exception:
            continue
    if not bldg_loops:
        continue

    # compute scale
    minx, miny, maxx, maxy = fixed_parcel.bounds
    height, width = maxx - minx, maxy - miny
    scale = max(height, width) + PAD * 2

    # blank canvases
    img_A = np.zeros((SIZE, SIZE), np.uint8)       # grayscale
    img_B = np.zeros((SIZE, SIZE, 3), np.uint8)    # BGR RGB

    # draw parcel: A=white; B=light gray
    for loop in parcel_loops:
        pts = np.array([
            [
                int((pt[0] - minx + PAD) / scale * 255),
                int((pt[1] - miny + PAD) / scale * 255)
            ]
            for pt in loop
        ], dtype=np.int32)
        cv2.fillPoly(img_A, [pts], 255)
        cv2.fillPoly(img_B, [pts], (200, 200, 200))

    # draw buildings: red (BGR=(0,0,255))
    for loop in bldg_loops:
        pts = np.array([
            [
                int((pt[0] - minx + PAD) / scale * 255),
                int((pt[1] - miny + PAD) / scale * 255)
            ]
            for pt in loop
        ], dtype=np.int32)
        cv2.fillPoly(img_B, [pts], (0, 0, 255))

    # save
    uid = f"beirut_{idx:05d}.png"
    cv2.imwrite(os.path.join(folder_A, uid), img_A)
    cv2.imwrite(os.path.join(folder_B, uid), img_B)
    meta.append(uid)

print(f"✅ Wrote {len(meta)} A/B PNG pairs → {folder_A} & {folder_B}")


Rasterising: 100%|██████████| 21143/21143 [01:18<00:00, 270.02it/s]

✅ Wrote 18096 A/B PNG pairs → data/processed/beirut_pix2pix\A & data/processed/beirut_pix2pix\B


In [ ]:
import os
import shutil
import cv2
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# ─── Settings & Paths ─────────────────────────────────────────────────────────
PAD  = 5
SIZE = 256
RAW = "datasets/beirut_buildings"      # where .shp files live
OUT = "data/processed/beirut_pix2pix"           # final pix2pix root
os.makedirs(OUT, exist_ok=True)

# ─── load shapefiles ─────────────────────────────────────────────────────
parcels   = gpd.read_file(f"{RAW}/beirut_parcels.shp").to_crs(epsg=3857)
footprints= gpd.read_file(f"{RAW}/beirut_buildings.shp").to_crs(epsg=3857)
parcels_fp   = "{RAW}/beirut_parcels.shp"
footprints_fp = "{RAW}/beirut_buildings.shp"


# Output folders for A, B, and raw AB
folder_A      = "archigan/data/processed/beirut_pix2pix/parcel_footprint_A/train"
folder_B      = "archigan/data/processed/beirut_pix2pix/parcel_footprint_B/train"
folder_AB_raw = "archigan/data/processed/beirut_pix2pix/parcel_footprint_AB/raw"

# Final AB splits
base_AB        = "archigan/data/processed/beirut_pix2pix/parcel_footprint_AB"
folder_AB_train = os.path.join(base_AB, "train")
folder_AB_val   = os.path.join(base_AB, "val")
folder_AB_test  = os.path.join(base_AB, "test")

# Ensure output directories exist
for d in [folder_A, folder_B, folder_AB_raw,
          folder_AB_train, folder_AB_val, folder_AB_test]:
    os.makedirs(d, exist_ok=True)

# ─── Load vector layers ───────────────────────────────────────────────────────
parcels    = gpd.read_file(parcels_fp)
footprints = gpd.read_file(footprints_fp)

meta = []  # collect all tile IDs

# ─── Rasterize each parcel + clipped buildings ───────────────────────────────
for idx, parc in tqdm(parcels.iterrows(), total=len(parcels), desc="Rasterising"):
    geom = parc.geometry
    if geom is None or geom.is_empty:
        continue

    sel = footprints[footprints.geometry.intersects(geom)]
    if sel.empty:
        continue

    fixed_parcel = geom.buffer(0)
    # parcel exterior loops
    parcel_loops = []
    if isinstance(fixed_parcel, Polygon):
        parcel_loops.append(list(fixed_parcel.exterior.coords))
    else:
        for part in fixed_parcel.geoms:
            parcel_loops.append(list(part.exterior.coords))

    # clipped building loops
    bldg_loops = []
    for poly in sel.geometry:
        try:
            fixed_bldg = poly.buffer(0)
            clipped = fixed_bldg.intersection(fixed_parcel)
            if clipped.is_empty:
                continue
            if isinstance(clipped, Polygon):
                bldg_loops.append(list(clipped.exterior.coords))
            elif isinstance(clipped, MultiPolygon):
                for sub in clipped.geoms:
                    bldg_loops.append(list(sub.exterior.coords))
        except Exception:
            continue
    if not bldg_loops:
        continue

    # compute scale for world → pixel
    minx, miny, maxx, maxy = fixed_parcel.bounds
    scale = max(maxx - minx, maxy - miny) + PAD*2

    # blank masks
    img_A = np.zeros((SIZE, SIZE),      np.uint8)  # input mask (grayscale)
    img_B = np.zeros((SIZE, SIZE, 3),   np.uint8)  # target mask (BGR)

    # draw parcel
    for loop in parcel_loops:
        pts = np.array([
            [
                int((x - minx + PAD)/scale * 255),
                int((y - miny + PAD)/scale * 255)
            ] for x, y in loop
        ], dtype=np.int32)
        cv2.fillPoly(img_A, [pts], 255)
        cv2.fillPoly(img_B, [pts], (200, 200, 200))

    # draw buildings
    for loop in bldg_loops:
        pts = np.array([
            [
                int((x - minx + PAD)/scale * 255),
                int((y - miny + PAD)/scale * 255)
            ] for x, y in loop
        ], dtype=np.int32)
        cv2.fillPoly(img_B, [pts], (0, 0, 255))

    # save A, B, and raw AB
    uid = f"beirut_{idx:05d}.png"
    cv2.imwrite(os.path.join(folder_A,      uid), img_A)
    cv2.imwrite(os.path.join(folder_B,      uid), img_B)

    img_AB = np.hstack([
        cv2.cvtColor(img_A, cv2.COLOR_GRAY2BGR),
        img_B
    ])
    cv2.imwrite(os.path.join(folder_AB_raw, uid), img_AB)

    meta.append(uid)

print(f"✅ Generated {len(meta)} raw AB images → {folder_AB_raw}")

# ─── Split raw AB into 80/10/10 train/val/test ──────────────────────────────
train_val, test = train_test_split(meta, test_size=0.10,      random_state=42)
train,     val  = train_test_split(train_val, test_size=0.1111111, random_state=42)

for subset, uids, out_folder in [
    ("train", train, folder_AB_train),
    ("val",   val,   folder_AB_val),
    ("test",  test,  folder_AB_test),
]:
    # clear old contents if needed
    shutil.rmtree(out_folder)
    os.makedirs(out_folder, exist_ok=True)
    for uid in uids:
        shutil.copy(
            os.path.join(folder_AB_raw, uid),
            os.path.join(out_folder,      uid)
        )
    print(f"✅ {subset}: {len(uids)} images → {out_folder}")

# ─── Display an example AB tile correctly ───────────────────────────────────
if meta:
    sample = train[0] if train else meta[-1]
    img_path = os.path.join(folder_AB_train, sample)
    img_BGR  = cv2.imread(img_path)
    img_RGB  = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6,3))
    plt.imshow(img_RGB)
    plt.axis('off')
    plt.title(f"Example AB: {sample}")
    plt.show()


c:\Users\charb\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: datasets/beirut_buildings/beirut_buildings.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


DataSourceError: archigan/data/raw/beirut/parcels.shp: No such file or directory

In [17]:
#%matplotlib notebook
import glob
#import io
import os
import cv2
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from svglib.svglib import svg2rlg
from collections import defaultdict
from fiona.crs import from_epsg
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import numpy as np

from archigan.datalayer import Layer
from archigan.pipeline import ArchiPipeline
from archigan.boston import ParcelInputLayer, ParcelOutputLayer, parse_GIS_bostonbuildings_2016
from archigan.cvc_fp import FootprintInputLayer, RepartitionInputLayer, RepartitionOutputLayer, parse_CVC_FP_svg
# Install into current kernel’s env
%pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [18]:
path = 'datasets/boston_buildings/2016'
x, y = 800, 1000
parsed = parse_GIS_bostonbuildings_2016(path)
parsed = {f'boston_{(j + x):04d}': sample for j, sample in enumerate(parsed[x:y])}

directory = './tmp/'

print('N:', len(parsed))

#ParcelInputLayer.samples_to_imgs(parsed, directory)
ParcelOutputLayer.samples_to_imgs(parsed, directory)

Reading parcel data: datasets/boston_buildings/2016\Parcels_2016_Data_Full.shp


DataSourceError: datasets/boston_buildings/2016\Parcels_2016_Data_Full.shp: No such file or directory

In [ ]:
svgs = glob.glob('datasets/ImagesGT/*.svg')
classes = (
    'Door',
    'Window',
    'Room',
    'Wall',
    'Separation',
    'Parking',
)
parsed = {svg: parse_CVC_FP_svg(svg, classes) for svg in svgs}
directory = './tmp/'

FootprintInputLayer.samples_to_imgs(parsed, directory)
RepartitionInputLayer.samples_to_imgs(parsed, directory)
RepartitionOutputLayer.samples_to_imgs(parsed, directory)

In [ ]:
stages = (
    'ParcelInputLayer',
    'ParcelOutputLayer',
    'FootprintInputLayer',
    'RepartitionInputLayer',
    'RepartitionOutputLayer',
)
layers = [os.path.join('./prepared/parcel2floorplan_5layer', layer) for layer in stages]
stages = [(j - 1, j) for j in range(1, len(layers))]
stages.pop(1)
directory = './prepared/parcel2floorplan_3stage'

print(layers, stages, directory)
pipeline = ArchiPipeline(layers, stages)

In [ ]:
pipeline.setup_training(directory)

In [ ]:
import geopandas as gpd

df = gpd.read_file('datasets/boston_streets/Boston_Street_Segments.shp')
df.head(100)